In [1]:
import pandas as pd
from pandas import Series,DataFrame
from sklearn.naive_bayes import BernoulliNB
import numpy as np
import random
import operator

In [2]:
db = pd.read_csv('C:\\Users\\asus\\desktop\\data.csv')

In [3]:
db.groupby('Response').describe()

Age         BMI         Dbp      Dpfunc     Insulin  \
Response                                                                     
0        count  500.000000  500.000000  500.000000  500.000000  500.000000   
         mean    31.190000   30.304200   68.184000    0.429734   68.792000   
         std     11.667655    7.689855   18.063075    0.299085   98.865289   
         min     21.000000    0.000000    0.000000    0.078000    0.000000   
         25%     23.000000   25.400000   62.000000    0.229750    0.000000   
         50%     27.000000   30.050000   70.000000    0.336000   39.000000   
         75%     37.000000   35.300000   78.000000    0.561750  105.000000   
         max     81.000000   57.300000  122.000000    2.329000  744.000000   
1        count  268.000000  268.000000  268.000000  268.000000  268.000000   
         mean    37.067164   35.142537   70.824627    0.550500  100.335821   
         std     10.968254    7.262967   21.491812    0.372354  138.689125   
         min     21.000000    0.000000    0.000000    0.088000    0.000000   
         25%     28.000000   30.800000   66.000000    0.262500    0.000000   
         50%     36.000000   34.250000   74.000000    0.449000    0.000000   
         75%     44.000000   38.775000   82.000000    0.728000  167.250000   
         max     70.000000   67.100000  114.000000    2.420000  846.000000   

                  Pglucose        Preg        Skin  
Response                                            
0        count  500.000000  500.000000  500.000000  
         mean   109.980000    3.298000   19.664000  
         std     26.141200    3.017185   14.889947  
         min      0.000000    0.000000    0.000000  
         25%     93.000000    1.000000    0.000000  
         50%    107.000000    2.000000   21.000000  
         75%    125.000000    5.000000   31.000000  
         max    197.000000   13.000000   60.000000  
1        count  268.000000  268.000000  268.000000  
         mean   141.257463    4.865672   22.164179  
         std     31.939622    3.741239   17.679711  
         min      0.000000    0.000000    0.000000  
         25%    119.000000    1.750000    0.000000  
         50%    140.000000    4.000000   27.000000  
         75%    167.000000    8.000000   36.000000  
         max    199.000000   17.000000   99.000000

In [4]:
#Filling 0 values, which are null values in this dataset, with the average of that respective feature according to the Response class

db.loc[(db['Response'] == 0) & (db['BMI'] == 0.0), "BMI"] = 30.3
db.loc[(db['Response'] == 1) & (db['BMI'] == 0.0), "BMI"] = 35.1

db.loc[(db['Response'] == 0) & (db['Insulin'] == 0.0), "Insulin"] = 68.792
db.loc[(db['Response'] == 1) & (db['Insulin'] == 0.0), "Insulin"] = 100.335821

db.loc[(db['Response'] == 0) & (db['Dbp'] == 0.0), "Dbp"] = 68.184
db.loc[(db['Response'] == 1) & (db['Dbp'] == 0.0), "Dbp"] = 70.82467

db.loc[(db['Response'] == 0) & (db['Pglucose'] == 0.0), "Pglucose"] = 109.98
db.loc[(db['Response'] == 1) & (db['Pglucose'] == 0.0), "Pglucose"] = 141.257

db.loc[(db['Response'] == 0) & (db['Skin'] == 0.0), "Skin"] = 19.664
db.loc[(db['Response'] == 1) & (db['Skin'] == 0.0), "Skin"] = 22.164

In [5]:
#Creating new dataframe which will hold features which are buckets on the original features

data = DataFrame(np.zeros(768*24).reshape(768,24), columns = ['preg1','preg2','preg3','pg1','pg2','pg3','dbp1','dbp2','dbp3','skin1','skin2','skin3',
                                                             'insulin1','insulin2','insulin3','bmi1','bmi2','bmi3','dpf1','dpf2','dpf3','age1','age2','age3'])
data['Response'] = db['Response']

In [6]:
#Assigning bucket values to data

for i in range(0,768):
    if db.ix[i,'Preg']<=1:
        data.loc[i,'preg1'] = 1
    elif db.ix[i,'Preg']<=3:
        data.loc[i,'preg2'] = 1
    elif db.ix[i,'Preg']<=6:
        data.loc[i,'preg3'] = 1
    
    if db.ix[i,'Pglucose']<=99.75:
        data.loc[i,'pg1'] = 1
    elif db.ix[i,'Pglucose']<=117:
        data.loc[i,'pg2'] = 1
    elif db.ix[i,'Pglucose']<=141:
        data.loc[i,'pg3'] = 1
    
    if db.ix[i,'Dbp']<=64:
        data.loc[i,'dbp1'] = 1
    elif db.ix[i,'Dbp']<=72:
        data.loc[i,'dbp2'] = 1
    elif db.ix[i,'Dbp']<=80:
        data.loc[i,'dbp3'] = 1
    
    if db.ix[i,'Skin']<=19.664:
        data.loc[i,'skin1'] = 1
    elif db.ix[i,'Skin']<=23:
        data.loc[i,'skin2'] = 1
    elif db.ix[i,'Skin']<=32:
        data.loc[i,'skin3'] = 1
    
    if db.ix[i,'Insulin']<=68.792:
        data.loc[i,'insulin1'] = 1
    elif db.ix[i,'Insulin']<=100:
        data.loc[i,'insulin2'] = 1
    elif db.ix[i,'Insulin']<=127.25:
        data.loc[i,'insulin3'] = 1
    
    if db.ix[i,'BMI']<=27:
        data.loc[i,'bmi1'] = 1
    elif db.ix[i,'BMI']<=32:
        data.loc[i,'bmi2'] = 1
    elif db.ix[i,'BMI']<=36:
        data.loc[i,'bmi3'] = 1
    
    if db.ix[i,'Dpfunc']<=0.24375:
        data.loc[i,'dpf1'] = 1
    elif db.ix[i,'Dpfunc']<=0.3725:
        data.loc[i,'dpf2'] = 1
    elif db.ix[i,'Dpfunc']<=0.62650:
        data.loc[i,'dpf3'] = 1
    
    if db.ix[i, 'Age']<=24:
        data.loc[i,'age1'] = 1
    elif db.ix[i, 'Age'] <=29:
        data.loc[i,'age2']=1
    elif db.ix[i,'Age']<=41:
        data.loc[i,'age3']=1

In [7]:
#Shuffling data set

data = data.iloc[np.random.permutation(len(data)-1)]

In [8]:
#Splitting dataset into training and test set

train_indices = random.sample(range(0,len(data)),int(0.75*len(data)))
test_indices = list()
for i in range(0,len(data)):
    if i in train_indices:
        continue
    else:
        test_indices.append(i)

train_data = data.loc[train_indices]
test_data  = data.loc[test_indices]
train_data.reset_index(drop=True,inplace=True)
test_data.reset_index(drop=True,inplace=True)

In [9]:
#Filling any remaining null values in the feature columns

cols = list(data)
cols.pop()
cols.sort()

for col in cols:
    test_data[col] = test_data[col].fillna(0).astype(int)
    train_data[col] = train_data[col].fillna(0).astype(int)
test_data['Response'] = test_data['Response'].fillna(0).astype(int)
train_data['Response'] = train_data['Response'].fillna(0).astype(int)

In [10]:
train_data.groupby(train_data['Response']).describe()

preg1       preg2       preg3         pg1         pg2  \
Response                                                                     
0        count  366.000000  366.000000  366.000000  366.000000  366.000000   
         mean     0.349727    0.267760    0.234973    0.336066    0.316940   
         std      0.477536    0.443398    0.424562    0.473008    0.465921   
         min      0.000000    0.000000    0.000000    0.000000    0.000000   
         25%      0.000000    0.000000    0.000000    0.000000    0.000000   
         50%      0.000000    0.000000    0.000000    0.000000    0.000000   
         75%      1.000000    1.000000    0.000000    1.000000    1.000000   
         max      1.000000    1.000000    1.000000    1.000000    1.000000   
1        count  209.000000  209.000000  209.000000  209.000000  209.000000   
         mean     0.210526    0.172249    0.234450    0.047847    0.186603   
         std      0.408661    0.378503    0.424672    0.213955    0.390528   
         min      0.000000    0.000000    0.000000    0.000000    0.000000   
         25%      0.000000    0.000000    0.000000    0.000000    0.000000   
         50%      0.000000    0.000000    0.000000    0.000000    0.000000   
         75%      0.000000    0.000000    0.000000    0.000000    0.000000   
         max      1.000000    1.000000    1.000000    1.000000    1.000000   

                       pg3        dbp1        dbp2        dbp3       skin1  \
Response                                                                     
0        count  366.000000  366.000000  366.000000  366.000000  366.000000   
         mean     0.215847    0.308743    0.273224    0.245902    0.491803   
         std      0.411972    0.462607    0.446225    0.431210    0.500617   
         min      0.000000    0.000000    0.000000    0.000000    0.000000   
         25%      0.000000    0.000000    0.000000    0.000000    0.000000   
         50%      0.000000    0.000000    0.000000    0.000000    0.000000   
         75%      0.000000    1.000000    1.000000    0.000000    1.000000   
         max      1.000000    1.000000    1.000000    1.000000    1.000000   
1        count  209.000000  209.000000  209.000000  209.000000  209.000000   
         mean     0.272727    0.157895    0.296651    0.263158    0.047847   
         std      0.446431    0.365518    0.457878    0.441405    0.213955   
         min      0.000000    0.000000    0.000000    0.000000    0.000000   
         25%      0.000000    0.000000    0.000000    0.000000    0.000000   
         50%      0.000000    0.000000    0.000000    0.000000    0.000000   
         75%      1.000000    0.000000    1.000000    1.000000    0.000000   
         max      1.000000    1.000000    1.000000    1.000000    1.000000   

                  ...           bmi1        bmi2        bmi3        dpf1  \
Response          ...                                                      
0        count    ...     366.000000  366.000000  366.000000  366.000000   
         mean     ...       0.314208    0.297814    0.199454    0.292350   
         std      ...       0.464835    0.457923    0.400137    0.455465   
         min      ...       0.000000    0.000000    0.000000    0.000000   
         25%      ...       0.000000    0.000000    0.000000    0.000000   
         50%      ...       0.000000    0.000000    0.000000    0.000000   
         75%      ...       1.000000    1.000000    0.000000    1.000000   
         max      ...       1.000000    1.000000    1.000000    1.000000   
1        count    ...     209.000000  209.000000  209.000000  209.000000   
         mean     ...       0.066986    0.258373    0.296651    0.215311   
         std      ...       0.250597    0.438791    0.457878    0.412025   
         min      ...       0.000000    0.000000    0.000000    0.000000   
         25%      ...       0.000000    0.000000    0.000000    0.000000   
         50%      ...       0.000000    0.000000    0.000000    0.00000

In [11]:
#Computing appropriate parameter values for different columns based on response class

params_1 = list()
params_0 = list()
for col in cols:
    params_1.append(train_data.groupby(train_data['Response'])[col].describe()[1][1])
    params_0.append(train_data.groupby(train_data['Response'])[col].describe()[0][1])

In [12]:
#Computing the probability of the 2 classes in the data

phi_1 = (train_data.groupby(train_data['Response'])['age1'].describe()[1][0])/len(train_data)
phi_0 = (train_data.groupby(train_data['Response'])['age1'].describe()[0][0])/len(train_data)

In [13]:
#Sorting indices of test_data

resp = test_data['Response']
test_data = test_data.drop('Response',axis=1).sort_index(axis=1)
test_data['Response'] = resp

In [14]:
#User-defined product function prod(), since python doesn't have a built in one to find product of elements in an array

def prod(iterable):
    return reduce(operator.mul, iterable, 1)

In [15]:
#Making predictions based on test data features

predict=list()
for i in range(0,len(test_data)):
    test_p1=[sum(x) for x in zip(test_data.ix[i],params_1)]
    test_p0=[sum(x) for x in zip(test_data.ix[i],params_0)]
    
    for j in range(0, len(test_p1)):
        if test_p1[j]>1:
            test_p1[j]-=1
        elif test_p1[j]<1:
            test_p1[j] = 1-test_p1[j]
        
        if test_p0[j]>1:
            test_p0[j]-=1
        elif test_p0[j]<1:
            test_p0[j]=1-test_p0[j]
            
    p1 = (prod(test_p1)*phi_1)/((prod(test_p1)*phi_1)+(prod(test_p0)*phi_0))
    p0 = (prod(test_p0)*phi_0)/((prod(test_p1)*phi_1)+(prod(test_p0)*phi_0))
    
    if p1 > p0:
        predict.append(1)
    else:
        predict.append(0)

In [16]:
#Checking performance of our classifier algorithm

count = 0.0
for i in range(0,len(test_data)):
    if test_data.ix[i,'Response']==predict[i]:
        count+=1
count
perform = count/len(test_data)
print 'Performance of classifier built from scratch:',perform

Performance of classifier built from scratch: 0.848958333333


In [17]:
#To run built-in classifier from sci-kit learn to compare performance

bnb = BernoulliNB()

train = train_data.drop('Response', axis=1)
test = test_data.drop('Response',axis=1)
pred = bnb.fit(train, train_data['Response']).predict(test)


count = 0.0
for i in range(0,len(test_data)):
    if test_data.ix[i,'Response']==pred[i]:
        count+=1
count
perform = count/len(test_data)
print 'Performance of sci-kit learn classifier:',perform

Performance of sci-kit learn classifier: 0.84375
